# Исследование поведения пользователей

``` Учебный проект ``` 

## Задача

Необходимо проверить у пользователей, *зарегистрировавшихся в 2018 году*:

- есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;

- различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.


---

## Вид и свойства исходных таблиц

In [81]:
import pandas as pd

In [82]:
# запишем исходные таблицы в переменные:
events_df = pd.read_csv('data/7_4_Events.csv')
purchase_df = pd.read_csv('data/purchase.csv')

In [83]:
# смотрим содержимое таблицы events_df
display(events_df)



,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
...,...,...,...,...,...,...
252329,281232,level_choice,hard,2020-07-02T10:02:15,NaN,87439
252330,281233,level_choice,medium,2020-07-02T11:38:52,NaN,87488
252331,281234,pack_choice,NaN,2020-07-02T11:42:14,NaN,87488
252332,281235,tutorial_start,NaN,2020-07-02T13:32:58,86127.0,87464


### Описание event_df
| Название столбца | Описание |
| --- | :--- |
id| идентификатор события
user_id|	уникальный идентификатор пользователя, совершившего событие в приложении
start_time|	дата и время события
event_type|	тип события (значения: registration — регистрация; tutorial_start — начало обучения; tutorial_finish — завершение обучения; level_choice — выбор уровня сложности; pack_choice — выбор пакетов вопросов)
tutorial_id|	идентификатор обучения (этот идентификатор есть только у событий обучения)
selected_level|	выбранный уровень сложности обучения

In [84]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


In [85]:
# смотрим содержимое таблицы purchase_df
display(purchase_df)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
...,...,...,...,...
5951,21625,87331,2020-07-06T09:02:07,50
5952,21626,87418,2020-07-06T14:16:37,100
5953,21627,87431,2020-07-06T22:48:59,50
5954,21628,87363,2020-07-07T05:38:56,100


### Описание purchase_df
| Название столбца | Описание |
| --- | :--- |
id	 | идентификатор события
user_id	| уникальный идентификатор пользователя, совершившего событие в приложении
event_datetime |	дата и время события/покупки
amount	| сумма оплаты

In [86]:
purchase_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


## Преобразование данных

In [87]:
# Небходимо привести столбец start_time к формату даты. Для этого удаляем строки с некорректными значениями. 
events_df = events_df.drop(index= [10981,229121], axis=0)

In [88]:
# Создаем функцию, чтобы убрать несуществующее 29 февраля 2017 года: 
def dates_for_drop(x):
    if x[0:10] == '2017-02-29':
        return '2017-02-29'
    else:
        return x
     
# Применяем функцию к столбцу start_time     
events_df['start_time'] = events_df['start_time'].apply(dates_for_drop)
events_df = events_df[events_df['start_time'] != '2017-02-29']

In [89]:
# Наконец, преобразуем столбец в формат даты
events_df['start_time'] =  pd.to_datetime(events_df['start_time'])

In [90]:
# Создаем список пользователей таблицы events_df, зарегистрировавшихся в 2018 году
mask1 = events_df['event_type'] == 'registration' #только регистрации
mask2 = events_df['start_time'].dt.year == 2018 #только 2018 год
ids = list(events_df[mask1&mask2]['user_id'])
len(ids)

19926

In [91]:
# Отбираем события, касающие только найденных пользователей.
events_df = events_df[events_df['user_id'].isin(ids)]

In [92]:
# Переведем столбец event_datetime в формат даты
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])
# Создаем список пользователей таблицы purchase_df, зарегистрировавшихся в 2018 году
purchase_df = purchase_df[purchase_df['user_id'].isin(ids)]

--- 

## Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты?


In [93]:

# Рассчитаем количество пользователей, выбравших определенный уровень: 
# Hard level
print('Количество пользователей, выбравших:')
hard_level_users = events_df[events_df['selected_level'] == 'hard'][
    'user_id'
].nunique()
print('hard level - ', hard_level_users)
# Medium level
medium_level_users = events_df[events_df['selected_level'] == 'medium'][
    'user_id'
].nunique()
print('medium level - ', medium_level_users)
# Easy level
easy_level_users = events_df[events_df['selected_level'] == 'easy'][
    'user_id'
].nunique()
print('easy level - ', easy_level_users)
# Уровень не выбран
nan_level_users = events_df['user_id'].nunique() - easy_level_users - medium_level_users - hard_level_users
print('без уровня - ', nan_level_users)

Количество пользователей, выбравших:
hard level -  1249
medium level -  4645
easy level -  2448
без уровня -  11584


In [94]:
# Запишем в переменную users_with_payment id оплативших пользователей, она потребуется для фильтрации.
users_with_payment = purchase_df['user_id']

# Рассчитаем количество оплативших пользователей, выбравших определенный уровень: 
print('Оплатившие пользователи, у которых:')
# Hard level
hard_level_users_series = events_df[
    (events_df['selected_level'] == 'hard')&(events_df['user_id'].isin(users_with_payment))][
        'user_id']
hard_level_users_with_payment = hard_level_users_series.nunique()
print('hard level - ', hard_level_users_with_payment)
# Medium level
medium_level_users_series = events_df[
    (events_df['selected_level'] == 'medium')&(events_df['user_id'].isin(users_with_payment))][
        'user_id']
medium_level_users_with_payment = medium_level_users_series.nunique()    
print('medium level - ',medium_level_users_with_payment)  
# Easy level 
easy_level_users_series = events_df[
    (events_df['selected_level'] == 'easy')&(events_df['user_id'].isin(users_with_payment))][
        'user_id']   
easy_level_users_with_payment = easy_level_users_series.nunique()  
print('easy level - ',easy_level_users_with_payment)  
# Not selected
nan_level_users_with_payment = events_df[events_df['user_id'].isin(users_with_payment)][
        'user_id'].nunique(
            ) - easy_level_users_with_payment - medium_level_users_with_payment - hard_level_users_with_payment
print('нет уровня - ',nan_level_users_with_payment, ', то есть выбор уровня является обязательным условием начала core игры')   

Оплатившие пользователи, у которых:
hard level -  442
medium level -  969
easy level -  189
нет уровня -  0 , то есть выбор уровня является обязательным условием начала core игры


In [95]:
# Рассчитаем процент оплативших пользователей среди всех пользователей в разрезе выбранного уровня сложности
print('Процент оплативших пользователей, среди пользователей выбравших:')
percent_hard_level_users = hard_level_users_with_payment / hard_level_users
print('Hard level - {:.2%}'.format(percent_hard_level_users))

percent_medium_level_users = medium_level_users_with_payment / medium_level_users
print('Medium level -{:.2%}'.format(percent_medium_level_users))

percent_easy_level_users = easy_level_users_with_payment / easy_level_users
print('Easy level -{:.2%}'.format(percent_easy_level_users))

Процент оплативших пользователей, среди пользователей выбравших:
Hard level - 35.39%
Medium level -20.86%
Easy level -7.72%


## Вывод 
### Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты?

#### **Чем выше сложность уровня, тем больше вероятность оплаты.** 

#### Исходя из этой закономерности, можно предположить, что увеличив количество пользователей, выбирающих hard level, можно повысить выручку игры.

--- 
## Различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности?

In [96]:
# Переименуем столбцы, чтобы избежать некорректного присоединения

events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id", "event_datetime": "purchase_time"})


In [97]:
# Из таблицы events_df берем только регистрации

registration_df = events_df[events_df['event_type'] == 'registration']

In [98]:
# Объединяем таблицы purchase_df и registration_df
merged_df = registration_df.merge(purchase_df, on='user_id', how='inner')
merged_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,purchase_time,amount
0,80336,registration,NaN,2018-01-02 01:35:56,NaN,27845,16845,2018-01-03 18:53:43,100
1,80418,registration,NaN,2018-01-03 11:14:57,NaN,27865,16846,2018-01-04 14:46:10,250
2,80466,registration,NaN,2018-01-04 11:50:43,NaN,27884,16854,2018-01-08 19:37:34,150
3,80539,registration,NaN,2018-01-05 10:45:33,NaN,27910,16849,2018-01-07 12:11:34,100
4,80540,registration,NaN,2018-01-05 10:48:24,NaN,27911,16848,2018-01-07 08:19:12,50


In [99]:
# Отсортируем таблицу на соответсвие переменнной hard_level_users_series 
hard_level = merged_df[merged_df['user_id'].isin(hard_level_users_series)]

# Найдем промежуток времени
hard_timedelta = (hard_level['purchase_time'] - hard_level['start_time']).describe()
print(hard_timedelta)

count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
dtype: object


In [100]:
# Отсортируем таблицу на соответсвие переменнной medium_level_users_series 
medium_level = merged_df[merged_df['user_id'].isin(medium_level_users_series)]

# Найдем промежуток времени
medium_timedelta = (medium_level['purchase_time'] - medium_level['start_time']).describe()
print(medium_timedelta)

count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
dtype: object


In [101]:
# Отсортируем таблицу на соответсвие переменнной easy_level_users_series 
easy_level = merged_df[merged_df['user_id'].isin(easy_level_users_series)]

# Найдем промежуток времени
easy_timedelta = (easy_level['purchase_time'] - easy_level['start_time'])
print(easy_timedelta.describe())

count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
dtype: object


## ВЫВОД
### Различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности?

Временной промежуток между регистрацией и оплатой у пользователей **не растет и не уменьшается с уровнем сложности**. Самой длительное среднее время (по медиане и среднему значению) у уровня medium, ярко выраженной зависимости не наблюдается. 